<a href="https://colab.research.google.com/github/kavyajeetbora/end_to_end_gee_with_python/blob/master/end_to_end_earth_engine/05_get_latest_imagery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import geemap
import ee
import datetime

ee.Authenticate()
ee.Initialize(project='kavyajeetbora-ee')

## Get the Area of interest

Use this [tool](https://boundingbox.klokantech.com/) to get the area of interest

In [2]:

geojson  = {
  "coordinates": [[[73.8374558931,15.7190197034],[73.8846627717,15.7190197034],[73.8846627717,15.7521479389],[73.8374558931,15.7521479389],[73.8374558931,15.7190197034]]],
  "type": "Polygon"
}
## Convert GeoJSON to ee.Geometry
geometry = ee.Geometry(geojson)

## Download the data

In [3]:
def mask_s2_clouds(image):
  """Masks clouds in a Sentinel-2 image using the QA band.

  Args:
      image (ee.Image): A Sentinel-2 image.

  Returns:
      ee.Image: A cloud-masked Sentinel-2 image.
  """
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloud_bit_mask = 1 << 10
  cirrus_bit_mask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = (
      qa.bitwiseAnd(cloud_bit_mask)
      .eq(0)
      .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
  )

  return image.updateMask(mask).divide(10000)


dataset = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterDate('2024-01-01', '2024-09-19')
    # Pre-filter to get less cloudy granules.
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
    .filter(ee.Filter.bounds(geometry))
    .map(mask_s2_clouds)
)

dataset = dataset.sort('system:index', False)
dataset.size()

## Visualization

In [4]:
def convert_to_yyyymmdd(date_string):
  try:
    date_part = date_string.split('_')[0]
    year = int(date_part[0:4])
    month = int(date_part[4:6])
    day = int(date_part[6:8])

    date_object = datetime.date(year, month, day)
    formatted_date = date_object.strftime('%Y-%m-%d')

    return formatted_date

  except ValueError:
    return None

# Example Usage
date_string = '20240423T052651_20240423T054254_T43PCT'
formatted_date = convert_to_yyyymmdd(date_string)

if formatted_date:
  print(f"Converted date: {formatted_date}")
else:
  print("Invalid date format")

Converted date: 2024-04-23


In [5]:
formatted_date

'2024-04-23'

In [6]:
image = dataset.median().clip(geometry)

In [7]:
visualization = {
    'min': 0.0,
    'max': 0.3,
    'bands': ['B4', 'B3', 'B2'],
}

m = geemap.Map()
m.centerObject(geometry, zoom=14)
m.add_layer(image, visualization, 'RGB')
m

Map(center=[15.73558464051677, 73.86105933240252], controls=(WidgetControl(options=['position', 'transparent_b…

In [8]:
# prompt: export the ee.image to tiff file

task = ee.batch.Export.image.toDrive(**{
    'image': dataset.first().clip(geometry),
    'description': f'exported_image_{formatted_date}',
    'folder': 'exported_images',
    'fileNamePrefix': 'exported_image',
    'region': geometry,
    'scale': 10,
    'maxPixels': 1e13,
    'fileFormat': 'GeoTIFF',
})
task.start()